In [1]:
import math
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

# Approximate KNN

In [85]:
#variable in this cell should be defined before
#set p=200 just for checking and the outputs are for checking
# p is the number of representative, default 1000
p = 200
distance='euclidean'
fea = np.random.randint(10,size=(200,2))
RpFea = fea[:,:]
N = fea.shape[0]
if p>N:
    p = N
Knn = 5

In [86]:
def litekmeans(X,K,MaxIter=100,Replicates=1,Start='random'):
    """   
    func: 
        partitions the points in the (N,p) matrix X into K clusters. This partition minimizes the sum
        of the within-cluster sums of point-to-cluster-centroid distance.
        
    arguments:
        'X' - the data matrix, size - (N,p), rows indicate the points, columns indicate the variables.
        'K' - the number of clusters.
        'MaxIter' Optional - Maximum number of iterations allowed.  Default is 100.
        'Replicates' Optional - Number of times to repeat the clustering, each with a new set of initial 
            centroids. Default is 1. If the initial centroids are provided, the replicate will be 
            automatically set to be 1.
        'Distance' - Distance measure, in P-dimensional space, that KMEANS should minimize with respect to.  
            Choices are: {'sqEuclidean'} - Squared Euclidean distance (the default), 'cosine' ignore.
        'Start' - Method used to choose initial cluster centroid positions, sometimes known as "seeds".  
            Choices are:
            {'sample'}  - Select K observations from X at random (the default),  'matrix' ignore.
    return:
        'label' -  an N-by-1 vector containing the cluster indices of each point. dim = 1
        'center' - the K cluster centroid locations in the K-by-P matrix center. dim = 2
    """
    km = KMeans(n_clusters=K,init=Start,max_iter=MaxIter).fit(X)
    center = km.cluster_centers_
    label = km.labels_
    return [label,center]

In [87]:
def distEucSq( X, Y ):
    Yt = np.transpose(Y)
    
    D = np.absolute(np.sum(X*X,1)[:, np.newaxis]+np.sum(Yt*Yt,0) - 2*np.dot(X,Yt))
    return D

def pdist2_fast( X, Y, metric):
    """
    func:
        Calculates the distance between sets of vectors.
    argument:
        'X' - matrix (N,d) N is the number of entries in dataset, d is the dimension of data.
        'Y' - matrix (K,d), K is number of rep-clusters.
        'metric' - the way to measure distanct.
    """
    
    if metric == 'sqeuclidean':
        D = distEucSq( X, Y )
    elif metric == 'euclidean':
        D = np.sqrt(distEucSq( X, Y ))
    else:
        D = None
    return D


## Step1: partition RpFea into rep-clusters

In [88]:
cntRepCls = math.floor(math.sqrt(p)) #number of rep-cluster
#print(cntRepCls)

14


## Step2: find the center of each rep-cluster

In [89]:
if (distance=='euclidean'):
    repClsLabel, repClsCenters = litekmeans(RpFea,cntRepCls,MaxIter=20);
else:
    repClsLabel, repClsCenters = litekmeans(RpFea,cntRepCls,MaxIter=20);
#print(repClsLabel)

[11  5  3  4  8 13  5  9  4 11  1  1  4  1 12 11  2  0 11 11  1  4  8  4
  1  6  9  8  3  6 13 12  3  2 11 11  9  8  2  2  9  3  8  7  6  2  2 12
 12  3 11  7  9  9  8  3  6  8  5 11  3  9  4  0  8 10  6  4  4 13 11  9
 13  4  0  3  3  8  9  7  6  4 10  5  3  6  0  2  6 11  9  7  8  6  2 10
  8 12  4 13  2  3  8 12  1  2 11  8 12  8  5  6  6  0 11  5  7  2  6  8
  1  0  6  4 11  8  7  0 12 11 11  6  9  0  1 10 11 12  8  0 11  3 13  4
  7  1 10  5  9  8  4  9 10 13 11 11  7  3 10  9 10  9  8 13  5  5  9  2
  8  6 12  2 13  9  2 11 12  7  8  0  3  3  4 13 13 11 11  9  2  0  5 11
  2  9  1 13 12  3  8  6]


## Step3: Pre-compute the distance between N objects and the rep-cluster

In [90]:
centerDist = pdist2_fast(fea, repClsCenters, distance);
#print(centerDist)

[[11.63529825  8.48999411  3.25060091 ...  1.15770367  9.49158984
   7.34090518]
 [ 6.1878313   8.48999411  5.4375     ...  7.48377875  5.14039017
   9.33928382]
 [ 3.27777389  2.80713377  6.47621851 ...  8.2768922   1.85030028
   4.71404521]
 ...
 [ 2.41551459  3.88329757  7.79848743 ...  9.66127723  2.14249336
   5.90668172]
 [ 8.3567165   7.42159013  1.75111572 ...  3.41666667  6.42315689
   7.31816613]
 [ 7.53887994  4.20475921  2.07759627 ...  3.31767154  5.36255018
   3.72677996]]


In [91]:
# Find the nearest rep-cluster (in RpFea) for each object
minCenterIdxs = np.argmin(centerDist,axis=1) # one dim
#print(minCenterIdxs)
# clear centerDist
cntRepCls = repClsCenters.shape[0]
#print(cntRepCls)                         

[11  5  3  4  8 13  5  9  4 11  1  1  4  1 12 11  2  0 11 11  1  4  8  4
  1  6  9  8  3  6 13 12  3  2 11 11  9  8  2  2  9  3  8  7  6  2  2 12
 12  3 11  7  9  9  8  3  6  8  5 11  3  9  4  0  8 10  6  4  4 13 11  9
 13  4  0  3  3  8  9  7  6  4 10  5  3  6  0  2  6 11  9  7  8  6  2 10
  8 12  4 13  2  3  8 12  1  2 11  8 12  8  5  6  6  0 11  5  7  2  6  8
  1  0  6  4 11  8  7  0 12 11 11  6  9  0  1 10 11 12  8  0 11  3 13  4
  7  1 10  5  9  8  4  9 10 13 11 11  7  3 10  9 10  9  8 13  5  5  9  2
  8  6 12  2 13  9  2 11 12  7  8  0  3  3  4 13 13 11 11  9  2  0  5 11
  2  9  1 13 12  3  8  6]
14


In [92]:
#Then find the nearest representative in the nearest rep-cluster for each object.
nearestRepInRpFeaIdx = np.zeros((N,1),dtype=np.int64) #here is 2 dim
  
for i in range(10):
    #cluster is from 0 to 9 including 9, in matlab 1:10 including 10
    #calculate the min index from fea which is nearest to cluster i to all rep in cluster i
    #nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)]=np.argmin(pdist2_fast(fea[np.where(minCenterIdx==i),:],RpFea[np.where(repClsLabel==i),:],distance),axis=1)
    # need to increase the dim of argmin
    #print(pdist2_fast(fea[np.where(minCenterIdxs==i)[0],:],RpFea[np.where(repClsLabel==i)[0],:],distance).shape)
    
    tmp = np.argmin(pdist2_fast(fea[np.where(minCenterIdxs==i)[0],:],RpFea[np.where(repClsLabel==i)[0],:],distance),axis=1)
    #print(tmp.shape)
    #print(nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)].shape)
    nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)] = np.expand_dims(tmp,axis=1)
    #the index of rep equals i
    tmp = np.where(repClsLabel==i)
    #print(np.squeeze(nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)],axis=1).shape)
    
    #invert offset index to real index of rep
    tmp[0][np.squeeze(nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)],axis=1)]
    nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)] = tmp[0][nearestRepInRpFeaIdx[np.where(minCenterIdxs==i)]]
#print(nearestRepInRpFeaIdx)
    #the result is 2 dim

(11, 11)
(10, 10)
(16, 16)
(16, 16)
(15, 15)
(10, 10)
(16, 16)
(9, 9)
(22, 22)
(19, 19)
[[  0]
 [  1]
 [  2]
 [  3]
 [  4]
 [  0]
 [  6]
 [  7]
 [  8]
 [  0]
 [ 10]
 [ 11]
 [  8]
 [ 10]
 [  0]
 [  0]
 [ 16]
 [ 17]
 [  0]
 [  0]
 [ 20]
 [ 21]
 [ 22]
 [ 23]
 [ 24]
 [ 25]
 [ 26]
 [ 27]
 [ 28]
 [ 29]
 [  0]
 [  0]
 [ 32]
 [ 33]
 [  0]
 [  0]
 [ 36]
 [  4]
 [ 38]
 [ 39]
 [ 40]
 [ 41]
 [ 42]
 [ 43]
 [ 44]
 [ 45]
 [ 16]
 [  0]
 [  0]
 [ 28]
 [  0]
 [ 51]
 [ 40]
 [ 40]
 [ 54]
 [ 41]
 [ 56]
 [ 27]
 [ 58]
 [  0]
 [ 60]
 [ 36]
 [  3]
 [ 63]
 [ 64]
 [  0]
 [ 66]
 [ 21]
 [ 21]
 [  0]
 [  0]
 [ 71]
 [  0]
 [ 23]
 [ 74]
 [ 32]
 [ 60]
 [ 77]
 [ 71]
 [ 43]
 [ 44]
 [ 23]
 [  0]
 [ 83]
 [ 28]
 [ 44]
 [ 74]
 [ 45]
 [ 25]
 [  0]
 [ 71]
 [ 91]
 [ 42]
 [ 44]
 [ 45]
 [  0]
 [ 27]
 [  0]
 [  8]
 [  0]
 [ 39]
 [ 41]
 [102]
 [  0]
 [ 20]
 [105]
 [  0]
 [ 54]
 [  0]
 [ 54]
 [ 83]
 [ 44]
 [ 44]
 [ 63]
 [  0]
 [ 83]
 [ 51]
 [ 38]
 [118]
 [ 54]
 [ 11]
 [121]
 [ 44]
 [ 21]
 [  0]
 [ 27]
 [ 43]
 [121]
 [  0]
 [  0]
 [

In [97]:
#For each object, compute its distance to the candidate neighborhood of its nearest representative not need to be 
# in one cluster(in RpFea)
neighSize = 10*Knn # The candidate neighborhood size. K' = knn*10
RpFeaW = pdist2_fast(RpFea,RpFea,distance) #distance matrix
RpFeaKnnIdx = np.argsort(RpFeaW,axis=1) #too long may
RpFeaKnnIdx = RpFeaKnnIdx[:,0:neighSize+1] #(p,K'+1)
#same method to nearestRepInRpFeaIdx
RpFeaKnnDist = np.zeros((N,RpFeaKnnIdx.shape[1])) #entry_i to K' distances and nearest rc
for i in range(p):
    #print(fea[np.where(nearestRepInRpFeaIdx==i)[0],:].shape)
    RpFeaKnnDist[np.where(nearestRepInRpFeaIdx==i),:] = pdist2_fast(fea[np.where(nearestRepInRpFeaIdx==i)[0],:], RpFea[RpFeaKnnIdx[i,:],:], distance)
#get full matrix for each entry with K' nearest reps in indices form
#select rows based on nearestRepInRpFeaIdx to create N * K' matrix
RpFeaKnnIdxFull = RpFeaKnnIdx[np.squeeze(nearestRepInRpFeaIdx,axis=1),:] #entry index corresponding to RpFeaKnnDist
    
#print(RpFeaKnnIdxFull)
    

[[  0  89  15 ... 125  98 102]
 [  1 110 147 ... 151 148 107]
 [  2  75  32 ...  63 194 179]
 ...
 [197 157  84 ...  91  24  53]
 [102 138 198 ... 136 164  34]
 [199  29 122 ...  35  30  50]]


In [98]:
#Get the final KNN according to the candidate neighborhood.
knnDist = np.zeros((N,Knn))
knnTmpIdx = np.zeros((N,Knn),dtype=np.int64)
knnIdx = np.zeros((N,Knn),dtype=np.int64)
for i in range(Knn):
    knnTmpIdx[:,i] = np.argmin(RpFeaKnnDist,axis=1)
    knnDist[:,i] = np.min(RpFeaKnnDist,axis=1)
    rowIdx = np.arange(N)
    RpFeaKnnDist[rowIdx,knnTmpIdx[:,i]] = 1e100  #set the accessed rep with large number
    knnIdx[:,i] = RpFeaKnnIdxFull[rowIdx,knnTmpIdx[:,i]] #mapping the index to rep cluster index which is nearest to the entry
#print(knnIdx)

[[  0  89  15  59 124]
 [  1 110 147   6  83]
 [  2  75  32  76  60]
 [150   3  62 199 182]
 [162   4  37  92  61]
 [  8  12  98 143  73]
 [  6   1  58 115  79]
 [166 151   7  91 161]
 [  8  12  98   3  68]
 [124 106   9 114 140]
 [134  13  10 145  24]
 [120  11 184  99  24]
 [  8  12  98   3  68]
 [134  13  10 145  24]
 [ 33 192  87  94  45]
 [  0  89  15  59 124]
 [ 16  46  38 188  34]
 [133  17 139 127 121]
 [ 18  19 154  35 130]
 [ 18  19 154  35 130]
 [ 20 194 104 142  24]
 [ 21  68 123  67  30]
 [ 22 168 107  77  54]
 [ 23  73  81 182  33]
 [ 24 145 104 194 184]
 [ 88  25 112 118 122]
 [ 26  90  78 132 159]
 [ 27 125  96  57 138]
 [197 157  84  49  28]
 [199  29 122 112 111]
 [  8  12  98 143  73]
 [ 87  94  45  33 192]
 [ 75  32 181   2 145]
 [192  33 199  29 174]
 [185 175  34 136 124]
 [ 18  19 154  35 130]
 [ 61  36 193 159  37]
 [162   4  37  92  61]
 [ 38 167 117  64 138]
 [100  39 171 118  94]
 [ 40  52  53 151  14]
 [ 41 101  55  84 128]
 [ 92 178  42 149  54]
 [ 43  79 1